codelab link: https://colab.research.google.com/github/abzzall/unsupervised_annotator1/blob/main/main.ipynb
github link: https://github.com/abzzall/unsupervised_annotator1.git

1. Extract multiword expression candidates. 
	1. Using the part-of-speech tags we extract multiword expression candidates, consisting of sequences of zero or more adjectives (ADJ followed by nouns (NOUN) or proper nouns (PROPNs) sequences.
	2. To generate training data for sequence tagging use sentence encoder like 
		a. EmbedRank (Bennani- Smires et al., 2018
		b. Key2Vec (Mahata et al., 2018)
    3. We implemented our Unsupervised Annotator using the POS tagger of SpaCy (Honnibal et al., 2020).

In [1]:
import re
!! pip install spacy 

['Requirement already satisfied: spacy in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (3.6.1)',
 'Requirement already satisfied: spacy-legacy<3.1.0,>=3.0.11 in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (from spacy) (3.0.12)',
 'Requirement already satisfied: spacy-loggers<2.0.0,>=1.0.0 in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (from spacy) (1.0.4)',
 'Requirement already satisfied: murmurhash<1.1.0,>=0.28.0 in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (from spacy) (1.0.9)',
 'Requirement already satisfied: cymem<2.1.0,>=2.0.2 in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (from spacy) (2.0.7)',
 'Requirement already satisfied: preshed<3.1.0,>=3.0.2 in c:\\users\\qyzyr\\onedrive\\documents\\pr

In [2]:
!! python -m spacy download en_core_web_sm
!!pip install sentence-transformers


['Requirement already satisfied: sentence-transformers in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (2.2.2)',
 'Requirement already satisfied: transformers<5.0.0,>=4.6.0 in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (from sentence-transformers) (4.33.0)',
 'Requirement already satisfied: tqdm in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (from sentence-transformers) (4.66.1)',
 'Requirement already satisfied: torch>=1.6.0 in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (from sentence-transformers) (2.0.1)',
 'Requirement already satisfied: torchvision in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (from sentence-transformers) (0.15.2)',
 'Requirement already satisfied: numpy in c:\\users\\qyzyr\\on

In [3]:
import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex
# Load the language model
nlp = spacy.load('en_core_web_sm')

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-mean-tokens')
from sentence_transformers import SentenceTransformer, util
from typing import List

In [4]:

# Define your text
with open('abstracts.txt') as file:
    text=file.read()


In [5]:
# text = "I love New York and San Francisco. Los Angeles is another great city."


In [6]:
from collections import namedtuple
CandidateMWE = namedtuple('CandidateMWE',['text','head', 'sentence','self_encode', 'sent_encode'])
CandidateW=namedtuple('CandidateW',['text','lemma', 'self_encode' ])
Term=namedtuple('Term',['text','detected' ])

In [ ]:
# Process the text with spaCy
doc = nlp(text)

# Extract MWEs (noun phrases) from the text
mwe_list = []
single_noun_list=[]
candidate_list=[]
for sent in doc.sents:
    sent_encode=model.encode(sent.text)
    for chunk in doc.noun_chunks:
        is_candidate=False
        word_count=0
        # mwe_list.append(chunk.text)
        if len(chunk.text.split()) > 1:
            noun_appeared=False
            is_candidate=True
            cleared_candidate=''
            for word in chunk:
                #IGNORING
                if word.pos_ in ['PUNCT', 'DET']:
                    continue
                elif word.pos_ not in ['ADJ', 'PROPN', 'NOUN']:
                    is_candidate=False
                    # print(f'{chunk.text} is not candidate 1 {word.text} -- {word.pos_}')
                    break
                elif word.pos_ in ['PROPN', 'NOUN']:
                    noun_appeared=True
                elif not(not noun_appeared and word.pos_=='ADJ'):
                    is_candidate=False
                    print(f'{chunk.text} is not candidate 2 {word.text} -- {word.pos_}, {noun_appeared}')
                    break
                cleared_candidate+=' '+word.text
                word_count+=1
        if is_candidate and word_count>1:
            cleared_candidate.strip()
            candidate_list.append(CandidateMWE(cleared_candidate, chunk.root.text, sent.text, model.encode(cleared_candidate), sent_encode))
            # print(cleared_candidate)
        else:
            single_noun_list +=[CandidateW(word.text, word.lemma_, model.encode(word.text)) for word in chunk if word.pos_ in ['NOUN', 'PROPN']]
                    
mwe_list=candidate_list+single_noun_list
# print(mwe_list)


context aware expectation maximization is not candidate 2 aware -- ADJ, True
label-rich source data is not candidate 2 rich -- ADJ, True
image-level weak labels is not candidate 2 weak -- ADJ, True
category-wise domain alignment is not candidate 2 wise -- ADJ, True
context aware expectation maximization is not candidate 2 aware -- ADJ, True
label-rich source data is not candidate 2 rich -- ADJ, True
image-level weak labels is not candidate 2 weak -- ADJ, True
category-wise domain alignment is not candidate 2 wise -- ADJ, True
context aware expectation maximization is not candidate 2 aware -- ADJ, True
label-rich source data is not candidate 2 rich -- ADJ, True
image-level weak labels is not candidate 2 weak -- ADJ, True
category-wise domain alignment is not candidate 2 wise -- ADJ, True
context aware expectation maximization is not candidate 2 aware -- ADJ, True
label-rich source data is not candidate 2 rich -- ADJ, True
image-level weak labels is not candidate 2 weak -- ADJ, True
cate

In [ ]:
def dist(wi_encode, wj_encode)->float:
    return util.pytorch_cos_sim(
        wi_encode,
        wj_encode
    )


def calculate_topic_score(expression_embedding, sentence_embedding)->float:
    """
    Calculate the topic score between a multiword expression and a sentence.

    Args:
        multiword_expression (str): The multiword expression.
        sentence (str): The sentence containing the expression.

    Returns:
        float: The topic score (cosine similarity) between the two embeddings.
    """
    # Load the distilbert-base-nli-mean-tokens model

    # Encode the multiword expression and sentence into embeddings
    # expression_embedding = model.encode(multiword_expression, convert_to_tensor=True)
    # sentence_embedding = model.encode(sentence, convert_to_tensor=True)

    # Calculate cosine similarity between the two embeddings
    similarity_score = util.pytorch_cos_sim(expression_embedding, sentence_embedding)

    # Extract the cosine similarity value from the tensor
    topic_score = similarity_score[0].item()

    return topic_score


def calculate_specificity_score(mw:CandidateMWE, w:List[CandidateW|CandidateMWE])->float:
    """
    Calculate the specificity score (SP) between a multiword expression (mw) and a list of words/multiword expressions (w).

    Args:
        mw (str): The multiword expression.
        w (list of str): The list of words/multiword expressions in the context.

    Returns:
        float: The specificity score (SP).
    """
    # Load the distilbert-base-nli-mean-tokens model
    # Calculate distances between mw and each word/phrase in w
    distances = [dist(mw.self_encode, wi.self_encode) for wi in w if wi.text != mw.text]

    # Calculate the mean of the distances
    specificity_score = sum(distances) / len(w)

    return specificity_score

def calculate_specifity_score_context_itself(mw:CandidateMWE):
    candidate_embedding=mw.self_encode
    s=0
    for word in mw.text.split():
        word_embedding=model.encode(word, convert_to_tensor=True)
        distance=dist(candidate_embedding, word_embedding)
        s+=distance
    return s/len(mw.text.split())

def calculate_specifity_score_context_sentence(mw:CandidateMWE):
    candidate_embedding=mw.self_encode
    s=0
    c=0
    sentence=mw.sentence
    doc=nlp(sentence)
    for word in doc.noun_chunks:
        word_embedding=model.encode(word, convert_to_tensor=True)
        distance=dist(candidate_embedding, word_embedding)
        s+=distance
        c+=1
    return s/len(c)




In [ ]:
TSP = 0.05
Ttopic = 0.1

In [ ]:
from datetime import datetime

First Variant

In [ ]:
term_mws1=[]

In [ ]:
start_time=datetime.now()
for candidate in candidate_list:
    topic_score=calculate_topic_score(candidate.self_encode, candidate.sent_encode)
    sp_score=calculate_specificity_score(candidate, mwe_list)
    if topic_score>Ttopic and sp_score>TSP:
        term_mws1.append(Term(candidate.text, 'by_score'))
        print(candidate.text, topic_score, sp_score)
end_time=datetime.now()
delta=end_time-start_time
print(f'duration: {delta.seconds}')

Second Variant. Filtering out by topic score first

In [ ]:
term_mws2=[]

In [ ]:
start_time=datetime.now()

temp_candidate=[]
for candidate in candidate_list:
    topic_score=calculate_topic_score(candidate.self_encode, candidate.sent_encode)
    # sp_score=calculate_specificity_score(candidate, mwe_list)
    if topic_score>Ttopic:
        temp_candidate.append(candidate)
for candidate in temp_candidate:
    # topic_score=calculate_topic_score(candidate.self_encode, candidate.sent_encode)
    sp_score=calculate_specificity_score(candidate, temp_candidate)
    if sp_score>TSP:
        term_mws2.append(Term(candidate.text, 'by_score'))        
        print(candidate.text, sp_score)
end_time=datetime.now()
delta=end_time-start_time
print(f'duration: {delta.seconds}')

Third Variant: Context is itself

In [ ]:
term_mws3=[]

In [ ]:
start_time=datetime.now()

for candidate in candidate_list:
    topic_score=calculate_topic_score(candidate.self_encode, candidate.sent_encode)
    sp_score=calculate_specifity_score_context_itself(candidate)
    if topic_score>Ttopic and sp_score>TSP:
        term_mws3.append(Term(candidate.text, 'by_score'))
        print(candidate.text, topic_score, sp_score)
end_time=datetime.now()
delta=end_time-start_time
print(f'duration: {delta.seconds}')

Forth Variant: Context is sentence

In [ ]:
term_mws3 = []


In [ ]:
start_time=datetime.now()

for candidate in candidate_list:
	topic_score = calculate_topic_score(candidate.self_encode, candidate.sent_encode)
	sp_score = calculate_specifity_score_context_sentence(candidate)
	if topic_score > Ttopic and sp_score > TSP:
		term_mws3.append(Term(candidate.text, 'by_score'))
		print(candidate.text, topic_score, sp_score)
end_time=datetime.now()
delta=end_time-start_time
print(f'duration: {delta.seconds}')

3. Upgrade single nouns according to morphological features.
	1. At this stage, we could have nouns that are not part of any multiword expressions, but still relevant.
	2. Check if the lemma of the noun is the same as any of the heads of the multiword expressions.
		a. Yes: we upgrade the noun to term 
		b. No: segment the word using a subword-unit segmentation and a vocabulary trained over a large general purpose corpus.
we use the vocabulary of the BERT-base model from HuggingFace (Wolf et al., 2020) and the corresponding tokenizer.

In [ ]:
from transformers import BertTokenizer

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
term_nouns=[]
subtoken_threshold=4
for candidate in single_noun_list:
    #Check if the lemma of the noun is the same as any of the heads of the multiword expressions.
    is_term=False
    lemma_is_head=False
    for term_mw in term_mws:
        if term_mw.head==candidate.lemma:
            is_term=True
            break
    if is_term:
        term_nouns.append(Term(candidate.text, 'by_lemma'))
        continue
    #segment the word using a subword-unit segmentation and a vocabulary trained over a large general purpose corpus.
    subtokens=tokenizer.tokenize(candidate.text)
    if len(subtokens)>subtoken_threshold:
        term_nouns.append(Term(candidate.text, 'by_subtokens'))

In [ ]:
terms=term_mws+term_nouns
with open('out.txt', 'w') as out_file:
    for term in terms:
        out_file.write(f'"{term.text}" appended {term.detected}\n')
        print(f'"{term.text}" appended {term.detected}\n')
        